In [ ]:
import requests
import pandas as pd
import time
import numpy as np
import os
from dotenv import load_dotenv


In [ ]:
load_dotenv()
API_KEY = os.getenv("GOOGLE_MAPS_API_KEY")

In [ ]:
# Define API endpoints
NEARBY_SEARCH_URL = "https://places.googleapis.com/v1/places:searchNearby"
PLACE_DETAILS_URL = "https://places.googleapis.com/v1/places/"

In [ ]:
# Define the approximate bounding box for Egypt
lat_min, lat_max = 22.0, 32.0
lng_min, lng_max = 25.0, 34.5

# Define grid step sizes (in degrees)
lat_step = 0.5  # ~55 km per 0.5° of latitude
lng_step = 0.5  # ~55 km per 0.5° of longitude

# Define search radius in meters
radius = 50000  # 50 km

In [ ]:
# Generate grid centers for the searches
latitudes = np.arange(lat_min, lat_max, lat_step)
longitudes = np.arange(lng_min, lng_max, lng_step)
search_centers = [(lat, lng) for lat in latitudes for lng in longitudes]

In [ ]:
place_queries = [
    "restaurant",
    "cafe",
    "museum",
    "art_gallery",
    "tourist_attraction",
    "monument"
]

In [ ]:
# Define the fields to retrieve from the Nearby Search request
nearby_field_mask = "places.id"

# Define the fields to retrieve from the Place Details request
details_field_mask = "id,displayName,attributions,photos,addressComponents,adrFormatAddress,formattedAddress,location,plusCode,shortFormattedAddress,types,viewport,accessibilityOptions,businessStatus,containingPlaces,displayName,googleMapsLinks,googleMapsUri,iconBackgroundColor,iconMaskBaseUri,primaryType,primaryTypeDisplayName,pureServiceAreaBusiness,subDestinations,utcOffsetMinutes,currentOpeningHours,currentSecondaryOpeningHours,internationalPhoneNumber,nationalPhoneNumber,priceLevel,priceRange,rating,regularOpeningHours,regularSecondaryOpeningHours,userRatingCount,websiteUri"

In [ ]:
# List to store detailed place responses
all_places = []
details_requests_count = 0
MAX_DETAILS_REQUESTS = 10000


In [ ]:
def search_nearby_places(lat, lng, place_type):
    """Performs a Nearby Search request and returns places found."""
    global details_requests_count

    payload = {
        "includedTypes": [place_type],
        "maxResultCount": 20,  # Maximum allowed by API
        "locationRestriction": {
            "circle": {
                "center": {"latitude": lat, "longitude": lng},
                "radius": radius
            }
        }
    }
    headers = {
        "Content-Type": "application/json",
        "X-Goog-Api-Key": API_KEY,
        "X-Goog-FieldMask": nearby_field_mask
    }

    try:
        response = requests.post(NEARBY_SEARCH_URL, json=payload, headers=headers)
        response_data = response.json()
        return response_data.get("places", [])
    except Exception as e:
        print(f"Error fetching nearby places for ({lat}, {lng}): {e}")
        return []


In [ ]:
# Function to fetch place details
def get_place_details(place_id):
    """Fetches detailed information for a given place ID."""
    global details_requests_count

    if details_requests_count >= MAX_DETAILS_REQUESTS:
        return None

    url = f"{PLACE_DETAILS_URL}{place_id}"
    headers = {
        "Content-Type": "application/json",
        "X-Goog-Api-Key": API_KEY,
        "X-Goog-FieldMask": details_field_mask
    }

    try:
        response = requests.get(url, headers=headers)
        details_requests_count += 1
        return response.json()
    except Exception as e:
        print(f"Error fetching details for {place_id}: {e}")
        return None

In [ ]:

# Loop over grid centers and place types
for lat, lng in search_centers:
    for place_type in place_queries:
        print(f"Searching for '{place_type}' near ({lat}, {lng})...")

        # Perform Nearby Search
        places_found = search_nearby_places(lat, lng, place_type)

        for place in places_found:
            place_id = place.get("id")
            if place_id and details_requests_count < MAX_DETAILS_REQUESTS:
                details = get_place_details(place_id)
                if details:
                    all_places.append(details)

        time.sleep(2)  # Avoid hitting API rate limits

    if details_requests_count >= MAX_DETAILS_REQUESTS:
        break

print(f"Total details requests made: {details_requests_count}")

In [ ]:
unique_places = {}
for place in all_places:
    place_id = place.get("id")
    if place_id:
        unique_places[place_id] = place  # Ensures only the latest entry is kept

# Convert to list
unique_places_list = list(unique_places.values())

In [ ]:
# Convert responses to DataFrame
data = []
for place in unique_places_list:
    data.append({
        "place_id": place.get("id"),
        "display_name": place.get("displayName", {}).get("text"),
        "name": place.get("name"),
        "attributions": place.get("attributions"),
        "photos": place.get("photos"),
        "address_components": place.get("addressComponents"),
        "adr_format_address": place.get("adrFormatAddress"),
        "formatted_address": place.get("formattedAddress"),
        "latitude": place.get("location", {}).get("latitude"),
        "longitude": place.get("location", {}).get("longitude"),
        "plus_code": place.get("plusCode"),
        "short_formatted_address": place.get("shortFormattedAddress"),
        "types": ", ".join(place.get("types", [])),
        "viewport": place.get("viewport"),
        "accessibility_options": place.get("accessibilityOptions"),
        "business_status": place.get("businessStatus"),
        "containing_places": place.get("containingPlaces"),
        "google_maps_links": place.get("googleMapsLinks"),
        "google_maps_uri": place.get("googleMapsUri"),
        "icon_background_color": place.get("iconBackgroundColor"),
        "icon_mask_base_uri": place.get("iconMaskBaseUri"),
        "primary_type": place.get("primaryType"),
        "primary_type_display_name": place.get("primaryTypeDisplayName"),
        "pure_service_area_business": place.get("pureServiceAreaBusiness"),
        "sub_destinations": place.get("subDestinations"),
        "utc_offset_minutes": place.get("utcOffsetMinutes"),
        "current_opening_hours": place.get("currentOpeningHours"),
        "current_secondary_opening_hours": place.get("currentSecondaryOpeningHours"),
        "international_phone_number": place.get("internationalPhoneNumber"),
        "national_phone_number": place.get("nationalPhoneNumber"),
        "price_level": place.get("priceLevel"),
        "price_range": place.get("priceRange"),
        "rating": place.get("rating"),
        "regular_opening_hours": place.get("regularOpeningHours"),
        "regular_secondary_opening_hours": place.get("regularSecondaryOpeningHours"),
        "user_rating_count": place.get("userRatingCount"),
        "website_uri": place.get("websiteUri")
    })

# Convert to Pandas DataFrame
df_places = pd.DataFrame(data)

# Save to CSV
df_places.to_csv("egypt_places_unique.csv", index=False)
print("Unique places data saved to egypt_places_unique.csv")

# Display DataFrame
import ace_tools as tools
tools.display_dataframe_to_user(name="Unique Egypt Places Data", dataframe=df_places)